In [1]:
!pip install ..
!pip install rouge-score

Processing /mnt/HQ/uni/text-mining/multidocsum
  Created wheel for multidocsum: filename=multidocsum-0.0.1-py3-none-any.whl size=8429 sha256=1988f7f318a101fb70a90d04060d3480a5c81e1f42eff99cc0ab123e59fd8739
  Stored in directory: /tmp/pip-ephem-wheel-cache-aaq526b8/wheels/02/74/95/c7b282f469bd75d8476184f75ec79b255b390a90201a66b918
Successfully built multidocsum
  Attempting uninstall: multidocsum
    Found existing installation: multidocsum 0.0.1
    Uninstalling multidocsum-0.0.1:
      Successfully uninstalled multidocsum-0.0.1


In [2]:
from src.models.extracting import Extractor
from src.data.data_io import DataIO
import numpy as np
from glob import glob
import numpy as np

In [3]:
cluster_name = []
for path in glob('../data/interim/extract_content/*'):
    cluster_name.append(path.split('/')[-1])

In [4]:
%%time
my_sum = {}
gold_sum = {}
for name in cluster_name:
    params = {
        'data_path': f'../data/interim/extract_content/{name}',
        'stop_words_path': '../data/external/stopwords.txt', 
        'punc_path': '../data/external/punctuation.txt',
        'compression_rate': 0.05,
        'remove_redundacy': True,
        'wc': 1.0, 
        'wp': 1.0, 
        'wf': 1.0, 
        'n_centroid': 10, 
        'C_max': 10, 
        'ngram_range': (1, 1), 
        'max_features': 100,
        'func': np.dot
    }
    summary = Extractor.extract(**params)
    os.makedirs(f'../summaries/{name}', exist_ok=True)
    with open(f'../summaries/{name}/summary.txt', 'w') as f:
        f.write('\n'.join(summary))
        my_sum[name] = '\n'.join(summary)
        gold_sum[name] = DataIO.load(f'../data/raw/ViMs/summary/{name}')

CPU times: user 29min 37s, sys: 1.06 s, total: 29min 38s
Wall time: 30min


In [5]:
from rouge_score import rouge_scorer

In [6]:
rougen = ['rouge1', 'rouge2', 'rouge3', 'rouge4']
scorer = rouge_scorer.RougeScorer(rougen)

In [15]:
%%time
mean_0 = {}
for n in rougen:
    mean_0[n] = {}
    rn = mean_0[n]
    precision = []
    recall = []
    fmeasure = []
    for cluster in my_sum:
        precision.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].precision)
        recall.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].recall)
        fmeasure.append(scorer.score(gold_sum[cluster]['0.gold.txt'], my_sum[cluster])[n].fmeasure)
    rn['precision'] = np.mean(precision)
    rn['recall'] = np.mean(recall)
    rn['f-measure'] = np.mean(fmeasure)

CPU times: user 13.8 s, sys: 6.81 ms, total: 13.8 s
Wall time: 13.9 s


In [8]:
import pandas as pd
pd.DataFrame(mean_0)

,rouge1,rouge2,rouge3,rouge4
precision,0.629926,0.405517,0.276815,0.228640
recall,0.804274,0.520454,0.347108,0.282270
f-measure,0.664363,0.428040,0.290197,0.238402


In [11]:

mean_1 = {}
for n in rougen:
    mean_1[n] = {}
    rn = mean_1[n]
    precision = []
    recall = []
    fmeasure = []
    for cluster in my_sum:
        precision.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].precision)
        recall.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].recall)
        fmeasure.append(scorer.score(gold_sum[cluster]['1.gold.txt'], my_sum[cluster])[n].fmeasure)
    rn['precision'] = np.mean(precision)
    rn['recall'] = np.mean(recall)
    rn['f-measure'] = np.mean(fmeasure)

CPU times: user 15 s, sys: 13.2 ms, total: 15 s
Wall time: 15.2 s


In [10]:
pd.DataFrame(mean_1)

,rouge1,rouge2,rouge3,rouge4
precision,0.677139,0.421916,0.277088,0.222519
recall,0.783271,0.492738,0.320094,0.255440
f-measure,0.690738,0.430924,0.281784,0.225637
